In [1]:
%run ../../../ingestion/ingestion.ipynb
%run ../../../data\ quality/quality.ipynb
%run ../../../write/write.ipynb
%run ../../../alerts/webhook.ipynb

25/05/07 21:42:07 WARN Utils: Your hostname, lnb035652cps resolves to a loopback address: 127.0.1.1; using 192.168.1.172 instead (on interface wlp0s20f3)
25/05/07 21:42:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/07 21:42:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/07 21:42:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/07 21:42:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/05/07 21:42:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# Cria uma instância da classe WebhookTeams utilizando o arquivo de configuração localizado em ../../config/teams.json
webhook = WebhookTeams("../../../config/teams.json")

In [3]:
try:
    # Cria um leitor de CSV com delimitador de linha definido como ";"
    reader = CSVReader(line_delimiter=";")
    
    # Executa a leitura do arquivo CSV e retorna um DataFrame
    df = client_code(reader, "../../../layer/bronze/saude/dic_saude_med_enf_mun.csv")
except:
    # Em caso de erro durante a leitura, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de leitura!")


In [4]:
df.show()

+--------------+--------+--------------------+--------------------+----+
|      Vari�vel|    Tipo|           Descri��o|               Fonte| Ano|
+--------------+--------+--------------------+--------------------+----+
|      cod_ibge|num�rico|C�digo IBGE do Mu...|                IBGE|2020|
|           mun|   texto|   Nome do Municipio|                IBGE|2020|
|      prof_sus|num�rico|V�nculos empregat...|Minist�rio da Sa�...|2023|
|     prof_nsus|num�rico|V�nculos empregat...|Minist�rio da Sa�...|2023|
|      prof_tot|num�rico|V�nculos empregat...|Minist�rio da Sa�...|2023|
|        medenf|   texto|Categ�rico: M�dic...|Minist�rio da Sa�...|2023|
|prof_habitante|num�rico|Profissional por ...|Minist�rio da Sa�...|2023|
+--------------+--------+--------------------+--------------------+----+



In [5]:
try:
    # Cria uma instância da classe responsável por exportar em formato Parquet
    exporter = ParquetExport()

    # Exporta o DataFrame para o caminho local especificado
    exporter.export(df, "../../../layer/prata/saude/dic_saude_med_enf_mun/", destination="local")
except:
    # Em caso de erro durante a exportação, envia uma notificação via Teams
    webhook.send_teams_notification("Erro de escrita!")
